In [ ]:
import os
import numpy as np
import quantities as pq
import neo
from neo.io.neomatlabio import NeoMatlabIO
import neurotic
from neurotic._elephant_tools import CausalAlphaKernel, instantaneous_rate

In [ ]:
feeding_bouts_tape_nori = {
    ('JG07', 'Tape nori',  0):           ('IN VIVO / JG07 / 2018-05-20 / 002', [2718, 2755  ]), # 5 swallows
    ('JG08', 'Tape nori',  0):           ('IN VIVO / JG08 / 2018-06-21 / 002', [ 147,  208  ]), # 7 swallows, some bucket and head movement
    ('JG08', 'Tape nori',  1):           ('IN VIVO / JG08 / 2018-06-21 / 002', [ 664,  701  ]), # 5 swallows, large bucket movement
    ('JG08', 'Tape nori',  2):           ('IN VIVO / JG08 / 2018-06-21 / 002', [1451, 1477  ]), # 3 swallows, some bucket movement
    ('JG11', 'Tape nori',  0):           ('IN VIVO / JG11 / 2019-04-03 / 004', [1227, 1280  ]), # 5 swallows, inward food movements had very low amplitude
    ('JG12', 'Tape nori',  0):           ('IN VIVO / JG12 / 2019-05-10 / 002', [ 436,  465  ]), # 4 swallows
    ('JG12', 'Tape nori',  1):           ('IN VIVO / JG12 / 2019-05-10 / 002', [2901, 2937  ]), # 5 swallows
    ('JG14', 'Tape nori',  0):           ('IN VIVO / JG14 / 2019-07-29 / 004', [ 829,  870  ]), # 5 swallows
    
    ('JG08', 'Tape nori', '1 superset'): ('IN VIVO / JG08 / 2018-06-21 / 002', [ 659,  726.1]), # 9 swallows, large bucket movement, used for prior modeling attempts
}

In [ ]:
metadata = neurotic.MetadataSelector(file='../../data/metadata.yml')

for (animal, food, bout_index), (data_set_name, time_window) in feeding_bouts_tape_nori.items():
    
    # load the metadata
    metadata.select(data_set_name)
    
    # load the data, including performing spike detection
    blk = neurotic.load_dataset(metadata, lazy=False)
    
    # slice the larger dataset down to just this swallowing bout
    seg = blk.segments[0].time_slice(time_window[0]*pq.s, time_window[1]*pq.s)
    
    # create a new Neo Block which will be filled with data and then exported to MATLAB
    new_blk = neo.Block(name=f'{animal} {food} {bout_index} -- "{data_set_name}", {time_window}')
    new_blk.file_origin = metadata.abs_path('data_file')
    new_seg = neo.Segment()
    new_blk.segments.append(new_seg)
    
    # specify the kernel (alpha function) to convolve with each spike train
    # for transforming them into a continuous firing rate representation
    sigma = 1*pq.s
    kernel = CausalAlphaKernel(sigma)
    kernel_name = f'{kernel.__class__.__name__}({kernel.sigma.magnitude}*{kernel.sigma.dimensionality.string})'

    # compute the firing rate for each spike train and store it in the new Neo Block
    for st in seg.spiketrains:
        sampling_period = seg.analogsignals[0].sampling_period.rescale('s')
        firing_rate = instantaneous_rate(st, sampling_period, kernel)

        firing_rate_times = firing_rate.times.rescale('s')    # fix time units
        firing_rate_values = firing_rate.clip(0*pq.Hz, None)  # replace with 0 any negative values (caused by numerical imprecision)
        firing_rate_sig = neo.AnalogSignal(
            firing_rate_values,
            t_start=firing_rate.t_start.rescale('s'),
            sampling_rate=firing_rate.sampling_rate.rescale('Hz'),
            name = f'{st.name} firing rate, {kernel_name}',
            dtype='float32',
        )

        new_seg.analogsignals.append(firing_rate_sig)
        new_seg.spiketrains.append(st)
    
    # get the empirical force measurement and store it in the new Neo Block
    force_sig = next((sig for sig in seg.analogsignals if sig.name == 'Force'), None)
    new_seg.analogsignals.append(force_sig)
    
    # write the new Neo Block to a MAT file
    export_dir = 'spikes-firing-rates-and-forces'
    if not os.path.exists(export_dir):
        os.mkdir(export_dir)
    filename = os.path.join(export_dir, f'{animal} {food} {bout_index}.mat')
    w = NeoMatlabIO(filename)
    w.write_block(new_blk)